In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('punkt')
#nltk.download('punkt_tab')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# Step 1 — Load CSV
# Columns: "Job Title" and "Job Description"
df = pd.read_csv("job_descriptions.csv")

# Example bias terms and job description, should be replaced
bias_terms = ["dominerende", "aggressiv", "rockstjerne", "ungdommelig", "naturlig leder"]


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Bias detection system using contextual embedding + similarity

After applying the lexicon-based bias detection algorithm, it is valuable to enhance the detection process using a semantic model. By embedding both the sentences of the job descriptions and the predefined bias terms, we can leverage semantic similarity to identify potential bias in job descriptions. This is achieved by computing the cosine similarity between the sentence embeddings and the bias term embeddings.

For each sentence in a job description, the cosine similarity score is compared to a pre-defined threshold. If the score exceeds this threshold, the sentence is labeled as "potentially biased." The threshold is adjustable based on experimentation and evaluation, allowing flexibility in detecting bias at different sensitivity levels.

However, it's important to note a few limitations of this approach:

* Lack of Contextual Sensitivity: The model does not understand the context in which the bias terms are used. For example, if a job description mentions "young people," and the bias term list contains the word "youthful," the sentence will be flagged as biased based solely on the similarity to the term "youthful". This does not account for whether "youthful" refers to actual youth, or if it is used in a different context entirely.

* False Positives: Since the function compares word embeddings directly, there is a possibility that sentences may be wrongly flagged as biased even if they do not express discriminatory intent. For example, a sentence may contain the word "leader" in a neutral context, but if the bias term "natural leader" is part of the bias lexicon, the sentence might still be flagged. This emphasizes that the model cannot capture all nuances in language and context.

* Manual Verification: As a result of the above limitations, any sentence flagged as potentially biased should be manually reviewed. It is essential to determine whether the flagged sentence truly reflects a biased statement or if the word appears in a different context that is not discriminatory. The function’s role is to highlight potential bias; it cannot make final judgments regarding the presence of bias without human oversight.





In [ ]:
def context_analysis(df, bias_terms, title_col="title", desc_col="description", threshold=0.5):
    """
    Detects potential bias in job descriptions based on semantic similarity to bias terms.

    Args:
        df (DataFrame): A pandas DataFrame with columns "title" and "description".
        bias_terms (list of str): List of bias words/phrases to detect.
        threshold (float): Cosine similarity threshold to flag potential bias.

    Returns:
        results (list of dict): Each dict contains job title, biased sentence, matched terms, and score.
    """

    # Load sentence-transformer model
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []

    # Precompute embeddings for bias terms
    bias_embeddings = model.encode(bias_terms, convert_to_tensor=True)

    # Loop through each job description
    for idx, row in df.iterrows():
        title = row["title"]
        description = row["description"]

        # Skip missing descriptions
        if pd.isna(description) or description.strip() == "":
            continue

        # Split into sentences
        all_sentences = sent_tokenize(description)

        # Embed all sentences
        sentence_embeddings = model.encode(all_sentences, convert_to_tensor=True)

        # Compute cosine similarities: (num_sentences x num_bias_terms)
        cosine_scores = util.cos_sim(sentence_embeddings, bias_embeddings)

        # Check each sentence separately
        for i, sentence in enumerate(all_sentences):
            # Skip sentences too short to be meaningful
            if len(sentence.split()) < 5 or len(sentence) < 30:
                continue
            
            # Get the cosine similarity scores for the current sentence
            scores = cosine_scores[i]

            # Check if any score exceeds the threshold
            matched_indices = (scores > threshold).nonzero(as_tuple=True)[0]

            # If there are any matches, store the results
            if len(matched_indices) > 0:
                matched_terms = [bias_terms[j] for j in matched_indices]
                results.append({
                    "Job Title": title,
                    "Potential Bias in": sentence,
                    "Matched terms": matched_terms,
                    "Max Score": torch.max(scores).item()
                })

    return results


In [ ]:
results = context_analysis(df, bias_terms, title_col="Job Title", desc_col="Job Description")
# Print results
print("Results of Contextual Embedding and Similarity for Bias Detection:")
for job in results:
    print(f"🔹 Job Title: {job['Job Title']}")
    if job['Potential Bias in']:
        print(f"  - Potential Bias in: '{job['Potential Bias in']}'")
        print(f"  - Matched terms: {job['Matched terms']}")
        print(f"  - Max Score: {job['Max Score']}")
    else:
        print("  - No biased words found.")
    print("-" * 50)

🔹 Job Title: To juridiske chefer med strategisk overblik og med lyst til
  - Potential Bias in: 'Erfaring med ledelse – gerne strategisk ledelse og evne til at omsætte strategi til handling.'
  - Matched terms: ['naturlig leder']
  - Max Score: 0.5431661009788513
--------------------------------------------------
🔹 Job Title: Underviser søges til ZBC10 i Næstved
  - Potential Bias in: 'Vi tilbyder et job med: Mulighed for at gøre en forskel for unge mennesker.'
  - Matched terms: ['ungdommelig']
  - Max Score: 0.5460309982299805
--------------------------------------------------
🔹 Job Title: Erfaren sygeplejerske med lyst til nattevagt
  - Potential Bias in: 'En nærværende og tilgængelig leder der har fokus på din og fællesskabets trivsel og udvikling.'
  - Matched terms: ['naturlig leder']
  - Max Score: 0.595171332359314
--------------------------------------------------
🔹 Job Title: Motiveres du af relationsarbejde med unge? Bliv
  - Potential Bias in: 'Velkommen indenfor på afsnit 

## Sentence-level Bias Classifier
After identifying sentences in job descriptions that are labeled as biased based on the bias terms both directly using lexicon-based detection and semantically using the contextual embedding, the next step is to build a classifier that can recognize biased language even when specific bias terms or their synonyms do not appear directly in the text.

The primary goal of this classifier is to generalize beyond the predefined bias terms and detect subtle instances of bias that may not be captured by a lexicon-based approach. While lexicon-based detection rely on matching explicit words or phrases, a classifier can learn to recognize patterns and relationships in the language that indicate bias, regardless of whether the exact terms are used. This generalization ability allows the classifier to identify indirect forms of bias that might otherwise go unnoticed.

By using a logistic regression model, we can train it on the labeled sentences (biased vs. non-biased) and enable it to learn the features and relationships between the text and the bias labels. The advantage of using this model is that it can process a wider range of language, recognizing biased language that does not rely solely on a pre-defined set of bias terms, and thus improving the overall detection accuracy.

The goal of the classifier is to train it to recognize bias in sentences, even if these sentences do not explicitly contain pre-defined bias terms. By achieving this, the classifier will be able to determine if any sentences in a job description is potentially bias. 



In [43]:
# Example bias terms and job description, should be replaced
bias_terms_weak_label = ["dominerende", "aggressiv", "ekstrovert", "ansvarlig", "naturlig leder", "faglig"]

In [44]:
def label_data(df, bias_terms_weak_label, title_col="title", desc_col="description"):
    sentences = []
    labels = []

    # Loop through each job description
    for idx, row in df.iterrows():
        title = row[title_col]
        description = row[desc_col]

        # Skip missing descriptions
        if pd.isna(description) or description.strip() == "":
            continue

        # Correct: Split the WHOLE description into sentences
        for sentence in sent_tokenize(description):
            sentences.append(sentence)
            # Weak labeling: check if any bias term is in the sentence
            if any(bias_word in sentence.lower() for bias_word in bias_terms_weak_label):
                labels.append(1)
            else:
                labels.append(0)

    labeled_df = pd.DataFrame({'sentence': sentences, 'label': labels})
    return labeled_df


In [49]:
df_labeled = label_data(df, bias_terms_weak_label, title_col="title", desc_col="description")

def classify_bias(df_labeled):
    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(df_labeled['sentence'])
    y = df_labeled['label']

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)

    #Cross-validation of the classifier
    #scores = cross_val_score(clf, X, y, cv=5, scoring='f1')
    #cross_validation = {"F1 scores:", scores, "Mean F1:", scores.mean()}
    return classification_report(y_test, y_pred)

In [50]:
print("Potential Bias Results:")
print(classify_bias(df_labeled))

Potential Bias Results:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       769
           1       1.00      0.03      0.05        76

    accuracy                           0.91       845
   macro avg       0.96      0.51      0.50       845
weighted avg       0.92      0.91      0.87       845



In [ ]:
#Cross-validation of the classifier
scores = cross_val_score(clf, X, y, cv=5, scoring='f1')
print("F1 scores:", scores)
print("Mean F1:", scores.mean())